import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [29]:
from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Lambda
from keras.applications import inception_resnet_v2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

In [30]:
n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

# 打上标签
y[12500:] = 1

100%|██████████| 12500/12500 [00:56<00:00, 221.73it/s]


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [32]:
i_input_tensor = Input((img_height, img_width, 3))
# x_input_tensor = inception_resnet_v2.preprocess_input(i_input_tensor)	# 错误的处理
x_input_tensor = Lambda(inception_resnet_v2.preprocess_input)(i_input_tensor) # 建立模型预处理

# 搭建基础模型
base_model = inception_resnet_v2.InceptionResNetV2(input_tensor=x_input_tensor, weights='imagenet', include_top=False)

范例：不运行
from keras.callbacks import ModelCheckpoint

'''
如果验证损失下降， 那么在每个训练轮之后保存模型。
'''
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])

### 下面修改

In [33]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='weights_checkpint_040_770.hdf5', verbose=1, save_best_only=True)

### 下面修改

In [34]:
# 锁定所有神经网络层
for layers in base_model.layers:
    layers.trainable = False

i_output = GlobalAveragePooling2D()(base_model.output)
i_output = Dropout(0.4)(i_output) # 0.25 比 0.5 好
i_predictions = Dense(1, activation='sigmoid')(i_output)
model = Model(base_model.input, i_predictions)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, batch_size=20,     # defalt 16
	epochs=8, 	# defalt 5
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 145s 7ms/step - loss: 0.1498 - acc: 0.9483 - val_loss: 0.0870 - val_acc: 0.9720

Epoch 00001: val_loss improved from inf to 0.08695, saving model to weights_checkpint_040_770.hdf5
Epoch 2/8
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0962 - acc: 0.9630 - val_loss: 0.0604 - val_acc: 0.9834

Epoch 00002: val_loss improved from 0.08695 to 0.06042, saving model to weights_checkpint_040_770.hdf5
Epoch 3/8
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0908 - acc: 0.9653 - val_loss: 0.0498 - val_acc: 0.9876

Epoch 00003: val_loss improved from 0.06042 to 0.04976, saving model to weights_checkpint_040_770.hdf5
Epoch 4/8
20000/20000 [==============================] - 138s 7ms/step - loss: 0.0825 - acc: 0.9693 - val_loss: 0.0924 - val_acc: 0.9716

Epoch 00004: val_loss did not improve from 0.04976
Epoch 5/8
20000/20000 [====================

In [36]:
for i in range(len(model.layers)):
    print(model.layers[i].name, i)

input_3 0
lambda_3 1
conv2d_407 2
batch_normalization_407 3
activation_407 4
conv2d_408 5
batch_normalization_408 6
activation_408 7
conv2d_409 8
batch_normalization_409 9
activation_409 10
max_pooling2d_9 11
conv2d_410 12
batch_normalization_410 13
activation_410 14
conv2d_411 15
batch_normalization_411 16
activation_411 17
max_pooling2d_10 18
conv2d_415 19
batch_normalization_415 20
activation_415 21
conv2d_413 22
conv2d_416 23
batch_normalization_413 24
batch_normalization_416 25
activation_413 26
activation_416 27
average_pooling2d_3 28
conv2d_412 29
conv2d_414 30
conv2d_417 31
conv2d_418 32
batch_normalization_412 33
batch_normalization_414 34
batch_normalization_417 35
batch_normalization_418 36
activation_412 37
activation_414 38
activation_417 39
activation_418 40
mixed_5b 41
conv2d_422 42
batch_normalization_422 43
activation_422 44
conv2d_420 45
conv2d_423 46
batch_normalization_420 47
batch_normalization_423 48
activation_420 49
activation_423 50
conv2d_419 51
conv2d_421 52


### 下面

In [37]:
# 放开靠后部分网络层进行 fine tune 微调模型
for layer in model.layers[770:]:	# total 783 layers
    layer.trainable = True


from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [38]:
model.fit(X_train, y_train, batch_size=20, 	# first: 16
	epochs=8, 	# first: 5 
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 147s 7ms/step - loss: 0.0771 - acc: 0.9713 - val_loss: 0.0412 - val_acc: 0.9906

Epoch 00001: val_loss improved from 0.04976 to 0.04121, saving model to weights_checkpint_040_770.hdf5
Epoch 2/8
20000/20000 [==============================] - 134s 7ms/step - loss: 0.0812 - acc: 0.9697 - val_loss: 0.0406 - val_acc: 0.9908

Epoch 00002: val_loss improved from 0.04121 to 0.04058, saving model to weights_checkpint_040_770.hdf5
Epoch 3/8
20000/20000 [==============================] - 134s 7ms/step - loss: 0.0831 - acc: 0.9705 - val_loss: 0.0425 - val_acc: 0.9906

Epoch 00003: val_loss did not improve from 0.04058
Epoch 4/8
20000/20000 [==============================] - 134s 7ms/step - loss: 0.0755 - acc: 0.9723 - val_loss: 0.0406 - val_acc: 0.9908

Epoch 00004: val_loss did not improve from 0.04058
Epoch 5/8
20000/20000 [==============================] - 134s 7ms/step - loss: 0.0739 - acc

### 下面修改

In [39]:
model.save('modelInceptionResNet_transfer_Doup040_770layer.h5')

读取测试图片数据

In [40]:
X_test = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    j = i+1
    X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j), (img_height, img_width))

100%|██████████| 12500/12500 [00:28<00:00, 443.94it/s]


In [41]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

25000/25000 [==============================] - 130s 5ms/step


### 下面修改

In [42]:
import pandas as pd
df = pd.read_csv("sample_submission.csv")

for i in range(12500):
	df.set_value(i, 'label', y_pred[i])

df.to_csv('pred_changePredictCsvMethond_Drop040_770layer.csv', index=None)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
